In [1]:
import cv2
import numpy as np


The requirements for verilog module with the video

1. independent of exact formmat
2. must 


In [2]:
video_file = "video.mp4"

In [3]:
TOTAL_LINES = 525
TOTAL_ACTIVE_LINES = 480
TOTAL_PIXELS = 858
TOTAL_ACTIVE_PIXELS = 720
TOTAL_BLANKING_PIXELS = TOTAL_PIXELS - TOTAL_ACTIVE_PIXELS

preamble = [0xFF, 0x00, 0x00]
# https://pub.smpte.org/pub/st293/st0293-2003_stable2010.pdf, table 3
SAV_active = 0b10000000
EAV_active = 0b10011101
SAV_VBI = 0b10101011
EAV_VBI = 0b10110110

EAV_START = 2 * TOTAL_ACTIVE_PIXELS
SAV_START =  2 * TOTAL_PIXELS - 4

DEFAULT_Y = 16
DEFAULT_C = 128
# what is going on during vertical blanking?
# todo do for one frame

In [9]:

cap = cv2.VideoCapture(video_file)
global_frame = np.zeros((TOTAL_ACTIVE_LINES, TOTAL_ACTIVE_PIXELS, 3), np.uint8)

counter = 0
all_frames = []
while (1):
    ret, frame = cap.read()

    if cv2.waitKey(1) & 0xFF == ord('q') or ret == False:
        cap.release()
        cv2.destroyAllWindows()
        break

    frame = cv2.resize(frame, (720, 480))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)

    lines = np.zeros((TOTAL_LINES, 2 * TOTAL_PIXELS)).astype(np.uint8)
    # YCrCb 4:2:2
    Y = frame[:, :, 0]
    Cr = frame[:, ::2, 1]
    Cb = frame[:, ::2, 2]

    lines[:TOTAL_ACTIVE_LINES, 0:2 * TOTAL_ACTIVE_PIXELS:4] = Cr
    lines[:TOTAL_ACTIVE_LINES, 1:2 * TOTAL_ACTIVE_PIXELS:2] = Y
    lines[:TOTAL_ACTIVE_LINES, 2:2 * TOTAL_ACTIVE_PIXELS:4] = Cb

    lines[:, EAV_START: EAV_START + 3] = preamble
    lines[:, SAV_START: SAV_START + 3] = preamble

    lines[:TOTAL_ACTIVE_LINES, EAV_START + 3] = EAV_active
    lines[:TOTAL_ACTIVE_LINES, SAV_START + 3] = SAV_active
    
    lines[TOTAL_ACTIVE_LINES:, EAV_START + 3] = EAV_VBI
    lines[TOTAL_ACTIVE_LINES:, SAV_START + 3] = SAV_VBI
    
    lines[:, EAV_START + 3 + 1:SAV_START:2] = DEFAULT_C
    lines[:, EAV_START + 3 + 2:SAV_START:2] = DEFAULT_Y

    lines[TOTAL_ACTIVE_LINES:, 0:EAV_START:2] = DEFAULT_C
    lines[TOTAL_ACTIVE_LINES:, 1:EAV_START:2] = DEFAULT_Y

    lines = np.roll(lines, 4)
    lines = np.roll(lines, TOTAL_LINES - TOTAL_ACTIVE_LINES, 0)
    


    # cv2.imshow('frame', lines)
    lines = lines.flatten()
    all_frames.append(lines)
    

    counter += 1
    # cv2.imshow('frame',global_frame)

In [10]:
all_frames = np.array(all_frames).flatten()

In [11]:
len(all_frames) / 1000 ** 2

196.3962

dtype('uint8')

In [12]:
with open("video.bin", "wb") as fd:
    fd.write(all_frames)

In [ ]:
%%sql


In [4]:
with open("video.bin", "rb") as fd:
    all_frames_read =  fd.read()

In [5]:
all_frames_read = np.array([*all_frames_read])

In [5]:
all_frames_read.shape

(196396200,)

In [6]:
all_frames =  np.reshape(all_frames_read, (-1, TOTAL_LINES * 2 * TOTAL_PIXELS)).astype(np.uint8)


In [23]:
try:
    for frame in all_frames:
        frame = np.reshape(frame, (TOTAL_LINES, 2 * TOTAL_PIXELS))
        # frame = frame[:100, :100]
        # frame = cv2.resize(frame, (480, 480))
        cv2.imshow('frame', frame)
        if cv2.waitKey(33) & 0xFF == ord('q'):
            # cap.release()
            cv2.destroyAllWindows()
            break
finally:
    cv2.destroyAllWindows()
    


In [22]:
len(all_frames)

218